In [ ]:
# ============================================================
# 📘 Financial Report Analyzer — Day 32: Summarization + Analysis
# Dataset: Coffee Sales (Excel from Kaggle)
# Purpose: Summarize parsed financial data using Groq AI
# Author: Boss (Entrepreneur, AI Business Builder)
# ============================================================

# === 1️⃣ Install & Import Dependencies ======================
!pip install --quiet groq pandas openpyxl

from groq import Groq
import pandas as pd
import json

# === 2️⃣ Initialize Groq Client ==============================
# Replace YOUR_GROQ_API_KEY with your actual key from https://console.groq.com
client = Groq(api_key="Your GROQ_API_KEY")

# === 3️⃣ Load Parsed Financial Data ==========================
# Assumes you already created these from Days 29–31
daily = pd.read_csv("daily_sales_summary.csv")
coffee = pd.read_csv("coffee_sales_summary.csv")
payment = pd.read_csv("payment_breakdown.csv")

print("✅ Data successfully loaded!")
print(f"Daily summary shape: {daily.shape}")
print(f"Coffee summary shape: {coffee.shape}")
print(f"Payment breakdown shape: {payment.shape}")

# === 4️⃣ Prepare Data for LLM Context ========================
# Convert small tables into strings to include in the AI prompt
daily_str = daily.to_string(index=False)
coffee_top5 = coffee.head(5).to_string(index=False)
payment_str = payment.to_string(index=False)

# === 5️⃣ Create the Prompt ==================================
system_msg = """
You are an experienced AI financial analyst for a retail café business.
Your role is to review financial data and generate an executive summary
that includes the following:
1. Overall sales trends and performance trajectory.
2. Top and low-performing coffee products.
3. Payment method insights (cash vs card trends).
4. Strategic recommendations for improving revenue or operations.

The tone should be professional and concise (150–300 words).
"""

user_msg = f"""
### DAILY REVENUE DATA
{daily_str}

### TOP 5 COFFEE PRODUCTS
{coffee_top5}

### PAYMENT BREAKDOWN
{payment_str}
"""

# === 6️⃣ Call Groq API for Summarization ====================
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",   # Groq-hosted model
    messages=[
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ],
    temperature=0.2,    # Lower = more factual
    max_tokens=500
)

# === 7️⃣ Display Summary ====================================
summary = response.choices[0].message.content
print("\n================= EXECUTIVE SUMMARY =================")
print(summary)
print("======================================================\n")

# === 8️⃣ Save Executive Summary =============================
with open("financial_summary.txt", "w", encoding="utf-8") as f:
    f.write(summary)
print("✅ Executive summary saved to financial_summary.txt")

# === 9️⃣ Structured JSON Output for Dashboards ===============
# This prompt enforces valid JSON for seamless integration
json_prompt = """
You are a financial AI assistant.
Analyze the following café financial data and return ONLY valid JSON (no extra text).
The JSON must follow this structure exactly:

{
  "total_revenue": <float>,
  "top_products": [<list of top 3 products as strings>],
  "key_trends": "<summary of major trends>",
  "recommendations": "<concise actionable recommendations>"
}

Ensure the response starts with '{' and ends with '}'.
"""

json_response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "system", "content": json_prompt},
        {"role": "user", "content": user_msg}
    ],
    temperature=0,
    max_tokens=400
)

# === 🔍 Attempt JSON Parsing & Save ==========================
raw_json_output = json_response.choices[0].message.content.strip()

try:
    # Ensure only JSON is parsed
    start_idx = raw_json_output.find("{")
    end_idx = raw_json_output.rfind("}") + 1
    clean_json_str = raw_json_output[start_idx:end_idx]
    structured_output = json.loads(clean_json_str)

    print("\n📊 Structured Summary (JSON):")
    print(json.dumps(structured_output, indent=4))

    # Save JSON output for dashboard or BI integration
    with open("financial_summary.json", "w", encoding="utf-8") as jf:
        json.dump(structured_output, jf, indent=4)
    print("✅ JSON summary saved to financial_summary.json")

except Exception as e:
    print("⚠️ JSON parsing failed. Here's the raw output instead:\n")
    print(raw_json_output)
    print("\nError:", e)


✅ Data successfully loaded!
Daily summary shape: (381, 2)
Coffee summary shape: (8, 2)
Payment breakdown shape: (2, 2)

================= EXECUTIVE SUMMARY =================
**Executive Summary**

Our retail café business has demonstrated a fluctuating sales trend over the past year, with daily revenues ranging from $27.92 to $836.66. The overall performance trajectory is positive, with a notable increase in sales during peak months.

**Key Insights:**

1. **Top-performing coffee products:** Latte, Americano with Milk, and Cappuccino are our best-selling products, accounting for a significant portion of our revenue.
2. **Low-performing coffee products:** Hot Chocolate, while still a popular item, generates relatively lower sales compared to our top-selling coffee products.
3. **Payment method trends:** Card payments dominate our sales, accounting for approximately 97% of our revenue, while cash payments make up only 3%.

**Strategic Recommendations:**

1. **Optimize menu offerings:** C